# ETL Parte 2

Como primer paso, se llevó a cabo la carga de los datos en la plataforma en la nube Google Cloud Platform (GCP), seguida de la normalización de los datos. Posteriormente, una vez definidos los objetivos y alcances del proyecto, procedimos a realizar una extracción de datos para trabajar con la información relevante para nuestro análisis.

Durante esta fase, se realizaron consultas en BigQuery, aprovechando la robustez de esta herramienta para establecer relaciones entre tablas. A través de estas consultas, logramos extraer únicamente la información relevante que necesitaremos en las etapas posteriores del proyecto

### Importaciones

In [9]:
import pandas as pd
import json
from textblob import TextBlob
import matplotlib.pyplot as plt


A través de esta consulta en BigQuery, realizamos la fusión de la información de las tablas "business" y "review". También aplicamos filtros según la zona donde vamos a desarrollar el análisis y por fecha.

In [ ]:
consulta_merge_businness_review = """
SELECT
  bussines.business_id,
  bussines.name,
  bussines.categories,
  bussines.city,
  bussines.latitude,
  bussines.longitude,
  bussines.stars AS bussines_stars,
  bussines.review_count,
  bussines.state,
  review.text,
  review.date,
  review.stars AS review_stars,
  review.cool,
  review.funny,
  review.useful,
  review.review_id
FROM
  `theta-byte-412400.YELP_REVIEW.RAW_REVIEW` AS review
JOIN
  `theta-byte-412400.YELP_BUSINESS.RAW_BUSINNES` AS bussines
ON
  review.business_id = bussines.business_id
WHERE
  DATE(review.date) >= '2021-01-01'
  AND bussines.categories LIKE '%Hotel%'
  AND bussines.state IN ('CA', 'OR', 'WA', 'AZ', 'NV');
"""

exportamos el resultado de la consulta en formato JSON para trabajar con estos datos

In [7]:
ruta_archivo_json = '../Data/bq-results-20240130-200859-1706645377030.json'

# Cargar el archivo JSON en un DataFrame de Pandas
df = pd.read_json(ruta_archivo_json, lines=True)

# Obtener información sobre el número de filas y columnas en el DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4413 entries, 0 to 4412
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   business_id     4413 non-null   object             
 1   name            4413 non-null   object             
 2   categories      4413 non-null   object             
 3   city            4413 non-null   object             
 4   latitude        4413 non-null   float64            
 5   longitude       4413 non-null   float64            
 6   bussines_stars  4413 non-null   float64            
 7   review_count    4413 non-null   int64              
 8   state           4413 non-null   object             
 9   text            4413 non-null   object             
 10  date            4413 non-null   datetime64[ns, UTC]
 11  review_stars    4413 non-null   int64              
 12  cool            4413 non-null   int64              
 13  funny           4413 non-null   i

In [5]:
df.head()

,business_id,name,categories,city,latitude,longitude,bussines_stars,review_count,state,text,date,review_stars,cool,funny,useful,review_id
0,yi9udL5fcmBjwcBcygv51Q,Canal Place,"Shoe Stores, Fashion, Caterers, Shopping, Hote...",New Orleans,29.950999,-90.064996,3.5,112,CA,The Canal Place is full of upscale shopping. W...,2021-11-28 18:53:59+00:00,3,3,0,6,zOGcU9618uIIpFCurQz85g
1,DlU5bGy4imZ5JzZYoQkdSg,College Hunks Hauling Junk & Moving - St Louis,"Local Services, Packing Supplies, Home Cleanin...",Saint Louis,38.687635,-90.394208,4.0,139,CA,We had a great experience with College Hunks r...,2021-01-18 03:09:14+00:00,5,0,0,1,13XkWHVhguwZWHIibguVEA
2,9NfPTgy_L9X57ep_Wq13Iw,Condor Express Whale Watching,"Whale Watching Tours, Hotels & Travel, Event P...",Santa Barbara,34.408290,-119.691469,4.5,82,AZ,"Honestly, deserves more than 5 stars. I had lo...",2021-06-29 03:26:50+00:00,5,0,0,2,gHpNqTAH2FpPCBO6p_YtYA
3,DlU5bGy4imZ5JzZYoQkdSg,College Hunks Hauling Junk & Moving - St Louis,"Local Services, Packing Supplies, Home Cleanin...",Saint Louis,38.687635,-90.394208,4.0,139,CA,I contacted them today 10/2 for a move on 10/2...,2021-10-02 20:03:22+00:00,1,0,0,0,AWktCbWrHNaT9pMBcr5VcQ
4,UcUgyFueY2LyHe3Zhcuhkg,Best Western Suites Near Opryland,"Hotels, Event Planning & Services, Hotels & Tr...",Nashville,36.223499,-86.696996,2.5,48,NV,Stayed 10/13/2021 - 10/17/2021. Rooms 211 & 2...,2021-10-22 16:41:13+00:00,1,0,0,0,UFoXp5oL5P8iFAVnYuGejA


Como próximo paso, llevamos a cabo un análisis de sentimiento sobre las reseñas que los usuarios proporcionaron, con el objetivo de compararlo posteriormente con las calificaciones en estrellas que estos mismos usuarios otorgan al negocio. De esta manera, buscamos determinar cuál de estos criterios es más fidedigno para nuestra evaluación

In [11]:
# Inicializa la columna de análisis de sentimiento
df['sentiment_analysis'] = 1

# Aplica el análisis de sentimiento solo para reseñas presentes
mask = df['text'].notnull()
df.loc[mask, 'sentiment_analysis'] = df.loc[mask, 'text'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)

# Aplica la escala de sentimiento con 0, 1 y 2
df['sentiment_analysis'] = pd.cut(df['sentiment_analysis'], bins=[-float('inf'), -0.1, 0.1, float('inf')], labels=[0, 1, 2], include_lowest=True)

df.head()

C:\Users\mateo\AppData\Local\Temp\ipykernel_12692\2697653386.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.07475649 0.01428571 0.14096825 ... 0.21818182 0.44666667 0.23484848]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[mask, 'sentiment_analysis'] = df.loc[mask, 'text'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)


,business_id,name,categories,city,latitude,longitude,bussines_stars,review_count,state,text,date,review_stars,cool,funny,useful,review_id,sentiment_analysis
0,yi9udL5fcmBjwcBcygv51Q,Canal Place,"Shoe Stores, Fashion, Caterers, Shopping, Hote...",New Orleans,29.950999,-90.064996,3.5,112,CA,The Canal Place is full of upscale shopping. W...,2021-11-28 18:53:59+00:00,3,3,0,6,zOGcU9618uIIpFCurQz85g,1
1,DlU5bGy4imZ5JzZYoQkdSg,College Hunks Hauling Junk & Moving - St Louis,"Local Services, Packing Supplies, Home Cleanin...",Saint Louis,38.687635,-90.394208,4.0,139,CA,We had a great experience with College Hunks r...,2021-01-18 03:09:14+00:00,5,0,0,1,13XkWHVhguwZWHIibguVEA,1
2,9NfPTgy_L9X57ep_Wq13Iw,Condor Express Whale Watching,"Whale Watching Tours, Hotels & Travel, Event P...",Santa Barbara,34.408290,-119.691469,4.5,82,AZ,"Honestly, deserves more than 5 stars. I had lo...",2021-06-29 03:26:50+00:00,5,0,0,2,gHpNqTAH2FpPCBO6p_YtYA,2
3,DlU5bGy4imZ5JzZYoQkdSg,College Hunks Hauling Junk & Moving - St Louis,"Local Services, Packing Supplies, Home Cleanin...",Saint Louis,38.687635,-90.394208,4.0,139,CA,I contacted them today 10/2 for a move on 10/2...,2021-10-02 20:03:22+00:00,1,0,0,0,AWktCbWrHNaT9pMBcr5VcQ,1
4,UcUgyFueY2LyHe3Zhcuhkg,Best Western Suites Near Opryland,"Hotels, Event Planning & Services, Hotels & Tr...",Nashville,36.223499,-86.696996,2.5,48,NV,Stayed 10/13/2021 - 10/17/2021. Rooms 211 & 2...,2021-10-22 16:41:13+00:00,1,0,0,0,UFoXp5oL5P8iFAVnYuGejA,1


Categorizamos la columna "review_stars" asignando las etiquetas de 1 y 2 estrellas a un comentario negativo, 3 estrellas a un comentario neutro, y 4 y 5 estrellas a un comentario positivo. Posteriormente, comparamos esta clasificación con nuestro análisis de sentimiento, donde aplicamos también una categorización en la que 0 representa un comentario negativo, 1 un comentario neutro y 2 un comentario positivo

In [12]:
# Mapea los niveles de sentimiento a las categorías deseadas
sentiment_mapping = {0: 'negativo', 1: 'neutro', 2: 'positivo'}
df['sentiment_category'] = df['sentiment_analysis'].map(sentiment_mapping)

# Define las categorías para las estrellas
star_mapping = {1: 'negativo', 2: 'negativo', 3: 'neutro', 4: 'positivo', 5: 'positivo'}
df['stars_category'] = df['review_stars'].map(star_mapping)

In [13]:
# Selecciona las columnas que deseas mostrar
selected_columns = ['sentiment_category', 'stars_category', 'sentiment_analysis','review_stars','text']

# Muestra las primeras 20 filas de las columnas seleccionadas
df[selected_columns].head(5)

,sentiment_category,stars_category,sentiment_analysis,review_stars,text
0,neutro,neutro,1,3,The Canal Place is full of upscale shopping. W...
1,neutro,positivo,1,5,We had a great experience with College Hunks r...
2,positivo,positivo,2,5,"Honestly, deserves more than 5 stars. I had lo..."
3,neutro,negativo,1,1,I contacted them today 10/2 for a move on 10/2...
4,neutro,negativo,1,1,Stayed 10/13/2021 - 10/17/2021. Rooms 211 & 2...


Como se observan rápidamente algunas discrepancias entre los dos análisis, creamos una tabla de contingencias para evaluarlas.

In [14]:
# Crea la tabla de contingencia
contingency_table = pd.crosstab(df['sentiment_category'], df['stars_category'], margins=True, margins_name='Total')

# Renombra la columna 'All' a 'Total'
contingency_table.rename(columns={'All': 'Total'}, inplace=True)

# Visualiza la tabla de contingencia
print(contingency_table)

stars_category      negativo  neutro  positivo  Total
sentiment_category                                   
negativo                 491       9         7    507
neutro                   916     114       102   1132
positivo                 395     221      2158   2774
Total                   1802     344      2267   4413


Podemos observar que hay un total de 2763 coincidencias entre las dos columnas, de un total de 4413 filas. Se decide realizar una ponderación entre estas dos columnas para tomar en cuenta ambos análisis y llegar a un resultado más fidedigno.